# Day 1: The Tyranny of the Rocket Equation

In [1]:
import os
import unittest

In [2]:
filename = os.path.join(os.getcwd(),"d1_input")
with open(filename) as f:
    mass_inputs = [int(mass) for mass in f.readlines()]

In [3]:
def requiredFuel(mass):
    return mass // 3 -2

def totalFuel(mass):
    input_fuel = requiredFuel(mass)
    f = requiredFuel(input_fuel)
    total_fuel = input_fuel
    while f>0:
        total_fuel += f
        f = requiredFuel(f)
    return total_fuel

def fuelSum(mass_inputs,callback):
    return sum(map(callback,mass_inputs))

In [4]:
# Answer 1
fuelSum(mass_inputs,requiredFuel)

3405637

In [5]:
# Answer 2
fuelSum(mass_inputs,totalFuel)

5105597

# Day 2: 1202 Program Alarm

In [54]:
def run_program(intcode_program):
    for i in range(0,len(intcode_program),4):
        if intcode_program[i]==1:
            intcode_program[intcode_program[i+3]] = intcode_program[intcode_program[i+1]] + intcode_program[intcode_program[i+2]]
        elif intcode_program[i]==2:
            intcode_program[intcode_program[i+3]] = intcode_program[intcode_program[i+1]] * intcode_program[intcode_program[i+2]]
        elif intcode_program[i]==99:
            break
        else:
            raise ValueError(f"Something went wrong. Opcode is {intcode_program[i]} at index {i} for the current program {intcode_program}")
    return intcode_program


In [55]:
class Test(unittest.TestCase):
    def test(self):
        self.states = [([1,9,10,3,2,3,11,0,99,30,40,50],[3500,9,10,70,2,3,11,0,99,30,40,50]),
                ([1,0,0,0,99],[2,0,0,0,99]),
                ([2,3,0,3,99],[2,3,0,6,99]),
                ([2,4,4,5,99,0],[2,4,4,5,99,9801]),
                ([1,1,1,4,99,5,6,0,99],[30,1,1,4,2,5,6,0,99])]
        for (program,output) in self.states:
            self.assertEqual(run_program(program),output)

unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [56]:
filename = os.path.join(os.getcwd(),"d2_input")
with open(filename) as f:
    initial_program = [int(opcode) for opcode in f.read().split(",")]
    intcode_program = initial_program[:]
intcode_program[1]=12
intcode_program[2]=2

# Answer 1
run_program(intcode_program)[0]

False

In [58]:
def findInput(init_program):
    for noun in range(50):
        for verb in range(50):
            program = init_program[:]
            program[1]=noun
            program[2]=verb
            if run_program(program)[0] == 19690720:
                return noun,verb,100*noun+verb
    
    return None

# Answer 2
findInput(initial_program)

(40, 19, 4019)

# Day 3: Crossed Wires

In [2]:
import scipy.sparse as sparse
import numpy as np
from copy import copy,deepcopy
import math

In [3]:
filename = os.path.join(os.getcwd(),"d3_input")
wires = []
with open(filename) as f:
    str_wires = f.read().splitlines()

In [4]:
class Point:
    def __init__(self,i,j):
        self.i = i
        self.j = j

    def __str__(self):
        return f"Point({self.i},{self.j})"

    def manhattanDistance(self,other_point):
        return abs(self.j-other_point.j)+abs(self.i-other_point.i)

In [20]:
class Wire:
    def __init__(self,string,size=20):
        self.full_path = [(el[0],int(el[1:])) for el in string.replace("\n","").split(",")]
        self.size = size#00
        
        self.create_path()
        # print(str(self.sparse))
        print("")

    def create_path(self,limit_try=10):
        def fill_mat():
            self.central_port = Point(self.size//2,self.size//2)
            current_point = copy(self.central_port)
            # self.sparse = np.zeros((self.size,self.size))
            # self.sparse = sparse.bsr_matrix((self.size,self.size))
            row = [self.size//2]
            col = [self.size//2]
            for (direction,nb_steps) in self.full_path:
                if direction=="U":
                    # self.sparse[current_point.i-nb_steps:current_point.i,current_point.j]=1
                    t_row = [i for i in range(current_point.i-nb_steps,current_point.i)]
                    row += t_row
                    col +=[current_point.j]*len(t_row)
                    current_point.i-=nb_steps
                elif direction=="D":
                    # self.sparse[current_point.i:current_point.i+nb_steps+1,current_point.j]=1
                    t_row = [i for i in range(current_point.i,current_point.i+nb_steps+1)]
                    row += t_row
                    col +=[current_point.j]*len(t_row)
                    current_point.i+=nb_steps
                elif direction=="L":
                    # self.sparse[current_point.i,current_point.j-nb_steps:current_point.j]=1
                    t_col = [i for i in range(current_point.j-nb_steps,current_point.j)]
                    col += t_col
                    row +=[current_point.i]*len(t_col)                    
     
                    current_point.j-=nb_steps            
                elif direction=="R":
                    # self.sparse[current_point.i,current_point.j:current_point.j+nb_steps+1]=1
                    t_col = [i for i in range(current_point.j,current_point.j+nb_steps+1)]
                    col += t_col
                    row +=[current_point.i]*len(t_col)                       
                    current_point.j+=nb_steps
            
            val = [1]*len(row)
            print(f"{len(row), len(col), len(val)}")
            # self.sparse[self.central_port.i,self.central_port.j]=2
            self.sparse = sparse.csc_matrix((val,(row,col)),shape=(max(row)+1,max(col)+1))

        count_try = 0
        while count_try<limit_try:
            try:
                fill_mat()
                print("Wire path created.")
                count_try = limit_try
            except IndexError as e:
                self.size = int(self.size*1.5)
                print(f"{e}  Change to size {self.size}")
        if count_try >limit_try:
            raise IndexError("Couldn't create wire path, the matrix was too small")
        
    def wiresCross(self,other_wire):
        self.intersections = np.multiply(self.sparse,other_wire.sparse)
        return self.sparse.multiply(other_wire.sparse)
    

    def getClosestIntersectionDist(self,other_wire):
        self.wiresCross(other_wire)
        min_dist = math.inf
        coord = np.argwhere(self.intersections==1)
        for i in range(len(coord)):
            dist = self.central_port.manhattanDistance(Point(coord[i,0],coord[i,1]))
            if min_dist>dist:
                min_dist = dist
        return min_dist


    def __str__(self):
        return str(self.sparse)

In [21]:
str_wires = ["R8,U5,L5,D3","U7,R6,D4,L4"]
# str_wires = ["R75,D30,R83,U83,L12,D49,R71,U7,L72","U62,R66,U55,R34,D71,R55,D58,R83"]
# str_wires = ["R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51","U98,R91,D20,R16,D67,R40,U7,R15,U6,R7"]

def getMin(str_wires,size=100):
    wires = [Wire(wire,size=size) for wire in str_wires]
    max_size = max(wires[0].size,wires[1].size)
    print(f"Common max size: {max_size}")
    wires = [Wire(wire,size=max_size) for wire in str_wires]
    print(wires[0])
    return wires[0].getClosestIntersectionDist(wires[1])

getMin(str_wires,20)

(24, 24, 24)
Wire path created.

(24, 24, 24)
Wire path created.

Common max size: 20
(24, 24, 24)
Wire path created.

(24, 24, 24)
Wire path created.

  (10, 10)	2
  (10, 11)	1
  (10, 12)	1
  (5, 13)	2
  (6, 13)	1
  (7, 13)	1
  (8, 13)	1
  (10, 13)	1
  (5, 14)	1
  (10, 14)	1
  (5, 15)	1
  (10, 15)	1
  (5, 16)	1
  (10, 16)	1
  (5, 17)	1
  (10, 17)	1
  (5, 18)	1
  (6, 18)	1
  (7, 18)	1
  (8, 18)	1
  (9, 18)	1
  (10, 18)	1


ValueError: dimension mismatch